### ШАГ 0
Импортируем необходимые модули 

In [1]:
# pip install PrettyTable # устанавливаем модуль для вывода таблиц

In [2]:
import sqlite3 as sql
import csv
from prettytable import PrettyTable

### ШАГ 1
Скачивание таблиц и создание БД  

In [5]:
with sql.connect('Gubanov_transaction_for_HW_3.db') as con:
    cur = con.cursor()
    tr = cur.execute(f'CREATE TABLE IF NOT EXISTS transaction_bd (TRANSACTION_ID INTEGER NOT NULL,TX_DATETIME NUMERIC NOT NULL,CUSTOMER_ID INTEGER NOT NULL,TERMINAL_ID INTEGER NOT NULL,TX_AMOUNT REAL NOT NULL)')
    customer = cur.execute(f'CREATE TABLE IF NOT EXISTS customer_bd (CLIENT_ID INTEGER NOT NULL,START_DT NUMERIC NOT NULL,END_DT NUMERIC NOT NULL,CLIENT_NAME TEXT NOT NULL,YEAR_BIRTH TEXT NOT NULL)')

In [6]:
with open('/Users/mikhailgubanov/Yandex.Disk.localized/Course_of_data_storage_and_processing_systems/transactions_for_dz2.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Пропускаем заголовок
    for row in reader:
        cur.execute('''
            INSERT INTO transaction_bd (TRANSACTION_ID,TX_DATETIME, CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT)
            VALUES (?, ?, ?, ?, ?)
        ''',row) # 

# Сохраняем изменения и закрываем соединение
con.commit()


In [7]:
with open('/Users/mikhailgubanov/Yandex.Disk.localized/Course_of_data_storage_and_processing_systems/client_info.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Пропускаем заголовок
    for row in reader:
        new_row = []
        for item in row:
            new_row.extend(item.split(';'))
        cur.execute('''INSERT INTO customer_bd (CLIENT_ID,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH) 
        VALUES (?, ?, ?, ?, ?)
        ''',(new_row[4], new_row[0], new_row[1], new_row[2], new_row[3])) # 

# Сохраняем изменения и закрываем соединение
con.commit()
con.close()

### ШАГ 2


#### За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’

In [9]:
"""функция для подключения к БД и выполнения запроса"""

def db_connect(request):
    with sql.connect('Gubanov_transaction_for_HW_3.db') as con:
        cur = con.cursor()
        cur.execute(request)
    con.commit()
    return cur.fetchall() 

a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [19]:
part_1_request = """SELECT t.rank, c.CLIENT_NAME,  c.END_DT, t.SUM_AMOUNT 
from 
(SELECT  CUSTOMER_ID, sum(TX_AMOUNT) as SUM_AMOUNT, (rank() over (ORDER by sum(TX_AMOUNT)  DESC )) as rank
from transaction_bd
GROUP by CUSTOMER_ID) as t 
LEFT JOIN customer_bd as c on t.CUSTOMER_ID = c.CLIENT_ID
WHERE t.rank = 1 and c.END_DT > '2023-05-01'"""

result_1 = db_connect(part_1_request)

# Создаем таблицу и добавляем заголовки
table_1 = PrettyTable()
table_1.field_names = ['RANK',"CLIENT_NAME","END_DT", "TOTAL_AMOUNT"]

# Добавляем данные в таблицу
for row in result_1:
    table_1.add_row(row)

# Выводим таблицу
print(table_1)

+------+-------------+------------+-------------------+
| RANK | CLIENT_NAME |   END_DT   |    TOTAL_AMOUNT   |
+------+-------------+------------+-------------------+
|  1   |    Kylee    | 2999-12-31 | 786115.8700000002 |
+------+-------------+------------+-------------------+


b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’

In [23]:
part_2_request = """SELECT c.CLIENT_NAME,  c.END_DT, t.SUM_AMOUNT
from 
(SELECT  CUSTOMER_ID, sum(TX_AMOUNT) as SUM_AMOUNT, (rank() over (ORDER by sum(TX_AMOUNT)  ASC )) as rank
from transaction_bd
GROUP by CUSTOMER_ID) as t 
LEFT JOIN customer_bd as c on t.CUSTOMER_ID = c.CLIENT_ID
WHERE t.rank = 1 and c.END_DT > '2023-05-01'
ORDER by t.rank"""

result_2 = db_connect(part_2_request)

# Создаем таблицу и добавляем заголовки
table_2 = PrettyTable()
table_2.field_names = ["CLIENT_NAME","END_DT", "TOTAL_AMOUNT"]

# Добавляем данные в таблицу
for row in result_2:
    table_2.add_row(row)

# Выводим таблицу
print(table_2)

+-------------+------------+--------------------+
| CLIENT_NAME |   END_DT   |    TOTAL_AMOUNT    |
+-------------+------------+--------------------+
|     Kloe    | 2999-12-31 | 30.479999999999997 |
+-------------+------------+--------------------+


c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.



In [29]:
part_3_request = """SELECT SUM(TOTAL_AMOUNT) as TOTAL_AMOUNT from 
(SELECT c.CLIENT_ID, c.CLIENT_NAME, SUM(t.TX_AMOUNT) AS TOTAL_AMOUNT
FROM transaction_bd t
JOIN customer_bd c ON t.CUSTOMER_ID = c.CLIENT_ID
WHERE c.CLIENT_ID LIKE '111%' 
AND c.END_DT >= '2023-05-01'
AND date(t.TX_DATETIME) < '2023-04-01'
GROUP BY c.CLIENT_ID, c.CLIENT_NAME);"""

result_3 = db_connect(part_3_request)

# Создаем таблицу и добавляем заголовки
table_3 = PrettyTable()
table_3.field_names = ["TOTAL_AMOUNT"]

# Добавляем данные в таблицу
for row in result_3:
    table_3.add_row(row)

# Выводим таблицу
print(table_3)

+--------------------+
|    TOTAL_AMOUNT    |
+--------------------+
| 1230296.5600000003 |
+--------------------+


d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.

In [30]:
part_4_request = """SELECT c.YEAR_BIRTH, sum(t.TX_AMOUNT) as TOTAL_AMOUNT
 FROM transaction_bd t
 JOIN customer_bd c on t.CUSTOMER_ID = c.CLIENT_ID
 WHERE c.END_DT >= DATE('2023-05-01')
 GROUP by c.YEAR_BIRTH
 ORDER by c.YEAR_BIRTH desc
"""

result_4 = db_connect(part_4_request)

# Создаем таблицу и добавляем заголовки
table_4 = PrettyTable()
table_4.field_names = ["YEAR_BIRTH","TOTAL_AMOUNT"]

# Добавляем данные в таблицу
for row in result_4:
    table_4.add_row(row)

# Выводим таблицу
print(table_4)

+------------+--------------------+
| YEAR_BIRTH |    TOTAL_AMOUNT    |
+------------+--------------------+
|    2000    | 86327196.30000111  |
|    1999    | 1041506.559999999  |
|    1998    |  601675.009999999  |
|    1997    | 8479942.930000037  |
|    1996    | 1483252.5100000026 |
|    1995    | 6321199.640000003  |
|    1994    | 6244280.799999996  |
|    1993    | 5732058.129999989  |
|    1992    | 5251433.300000004  |
|    1991    | 7141073.570000012  |
|    1990    | 6617599.939999983  |
|    1989    | 6303724.260000009  |
|    1988    | 7471929.980000016  |
|    1987    |  6629354.17999999  |
|    1986    | 5026130.510000013  |
|    1985    |  6742343.15000002  |
|    1984    | 7277775.240000034  |
|    1983    | 339072.0199999999  |
|    1982    | 13709527.609999966 |
|    1981    | 5321969.969999997  |
|    1980    |  9232299.43999999  |
|    1979    | 5679293.410000042  |
|    1978    | 4994142.379999988  |
|    1977    | 10338469.520000035 |
|    1976    | 11666129.2399

e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

In [31]:
part_5_request = """SELECT c.CLIENT_ID, count(t.TRANSACTION_ID) as COUNT_TR
 FROM transaction_bd t
 JOIN customer_bd c on t.CUSTOMER_ID = c.CLIENT_ID
 WHERE c.END_DT >= DATE('2023-05-01')
 GROUP by t.CUSTOMER_ID
 ORDER by 2 DESC
"""

result_5 = db_connect(part_5_request)

# Создаем таблицу и добавляем заголовки
table_5 = PrettyTable()
table_5.field_names = ["CLIENT_ID","COUNT_TR"]

# Добавляем данные в таблицу
for row in result_5:
    table_5.add_row(row)

# Выводим таблицу
print(table_5)

+-----------+----------+
| CLIENT_ID | COUNT_TR |
+-----------+----------+
|    2891   |   477    |
|    2580   |   464    |
|    1953   |   455    |
|    2932   |   454    |
|    3864   |   453    |
|    1246   |   453    |
|    2470   |   452    |
|    4231   |   451    |
|    1906   |   451    |
|    379    |   451    |
|    4661   |   449    |
|    382    |   449    |
|    1288   |   446    |
|    4539   |   445    |
|    2404   |   444    |
|    4831   |   442    |
|    2680   |   442    |
|    149    |   441    |
|    2114   |   440    |
|    1411   |   440    |
|    1031   |   440    |
|    4168   |   439    |
|    3728   |   439    |
|    1249   |   439    |
|    288    |   439    |
|    148    |   439    |
|    775    |   437    |
|    4961   |   436    |
|    3651   |   436    |
|    4436   |   435    |
|    4050   |   435    |
|    2446   |   435    |
|    507    |   434    |
|    1693   |   433    |
|    1066   |   433    |
|    4964   |   432    |
|    4499   |   432    |


### ШАГ 3 

(Балл - 3) Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [34]:
part_6_request = """SELECT
        strftime('%m', TX_DATETIME) as MONTH_NUMBER,
        CASE strftime('%m', TX_DATETIME) 
            WHEN '01' then 'January' 
            WHEN '02' then 'Febuary' 
            WHEN '03' then 'March' 
            WHEN '04' then 'April' 
            ELSE '' 
            END as MONTH,
        sum(t.TX_AMOUNT) as TOTAL_AMOUNT
    FROM
        transaction_bd t
        JOIN customer_bd c on t.CUSTOMER_ID = c.CLIENT_ID
    WHERE c.END_DT >= DATE('2023-05-01')
    GROUP by strftime('%m', TX_DATETIME)
    ORDER by MONTH_NUMBER
"""

result_6 = db_connect(part_6_request)

# Создаем таблицу и добавляем заголовки
table_6 = PrettyTable()
table_6.field_names = ['MONTH_NUMBER',"MONTH","TOTAL_AMOUNT"]

# Добавляем данные в таблицу
for row in result_6:
    table_6.add_row(row)

# Выводим таблицу
print(table_6)

+--------------+---------+--------------------+
| MONTH_NUMBER |  MONTH  |    TOTAL_AMOUNT    |
+--------------+---------+--------------------+
|      01      | January | 115919886.26999806 |
|      02      | Febuary | 106229454.85999985 |
|      03      |  March  | 115053440.2700006  |
|      04      |  April  | 72618124.49000086  |
+--------------+---------+--------------------+
